# 프롬프트 (`Prompt`)
`02_prompt.ipynb`

- LLM한테 주는 입력(지시, 맥락, 기억)
    - 지시: '~~해줘'
    - 맥락: Context - 현재 지시를 위해 제공하는 추가 정보
    - 기억: Memory - 지금까지 했던 대화 내용

In [ ]:
from dotenv import load_dotenv

load_dotenv()

## `PromptTemplate`

- 단순히 1회성 명령을 내릴 때 사용

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

from langchain_core.prompts import PromptTemplate

llm = ChatOpenAI(model='gpt-4.1-nano')

In [ ]:
# 추후에 체인.invoke에서 {} 내부에 들어갈 말을 채워줘야 함.
template = '{country}의 수도가 어디인가요?'

# .from_template 메서드로 프롬프트 만들기
prompt = PromptTemplate.from_template(template)

# {}를 채우는 방법 (우리는 결국 chain.invoke 로 쓰게 됨)
prompt.format(country='대한민국')

In [ ]:
chain = prompt | llm

chain.invoke({'country': '대한민국'}).content

### Partial Variable (부분 변수)

- 프롬프트에서 매개변수 기본값 사용하기

In [ ]:
template = '{c1}과 {c2}의 수도는 각각 어디인가요?'

prompt = PromptTemplate(
    template=template,
    input_variables=['c1'],
    partial_variables={
        'c2': '미국'
    }
)

prompt.format(c1='한국')
# prompt.format(c2='캐나다') 

In [ ]:
chain = prompt | llm
chain.invoke({'c1': 'kor', 'c2': 'us'}).content

In [ ]:
from datetime import datetime

def get_today():
    return datetime.now().strftime('%b %d')

prompt = PromptTemplate(
    template='오늘 날짜는 {today}입니다. 오늘 생일인 유명인 {n}명을 생년월일과 함께 나열해 주세요',
    input_variables=['n'],
    partial_variables={
        'today': get_today
    }
)

prompt.format(n=3)

In [ ]:
llm = ChatOpenAI(model='gpt-5-nano')

chain = prompt | llm
# 오늘 날짜 기준
print(chain.invoke({'n': 3}).content)
print('===')
print(chain.invoke({'n': 3, 'today': 'Jan 2'}).content)

## `ChatPromptTemplate`
- 채팅을 주고받는 템플릿 생성용
- 대화 목록을 LLM에게 주입
- 하나의 Chat 은 `role` 과 `message` 로 구성됨

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template('{country}의 수도는 어디?')
chat_prompt.format(country='한국')

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [
        # role - message
        ('system', '당신은 친절한 AI어시스트. 이름은 {name} 야.'),
        ('human', '반가워!'),
        ('ai', '무엇을 도와드릴까요?'),
        ('human', '{user_input}')
    ]
)

# format vs format_messages
messages_str = chat_template.format(name='gaida', user_input='이름이 뭐니?')
messages_cls = chat_template.format_messages(name='gaida', user_input='이름이 뭐니?')
print(messages_str)
print(messages_cls)

In [ ]:
# 한덩어리 텍스트
llm.invoke(messages_str).content
# 실제 대화 내역으로 인지 (Langsmith 가서 확인)
llm.invoke(messages_cls).content

In [ ]:
chain = chat_template | llm | StrOutputParser()

chain.invoke({'name': '가이다', 'user_input': '내 이름은 유태영. 너와 나의 이름점을 봐줘'})

## 프롬프팅 방법
내가 원하는 답변을 구구절절 설명 X -> 예시를 주고 답변을 형태를 유도
- Zero-shot prompting : 예시 없이 질문만 -> 바로 답변
- One-shot prompting  : 예시 1개 + 질문 -> 모델이 예시를 모방해서 답변
- **Few-shot prompting**  : 예시 여러개 + 질문 -> 예시들의 패턴을 일반화 해서 답변

In [ ]:
from langchain_core.prompts.few_shot import FewShotPromptTemplate
from langchain_core.output_parsers import StrOutputParser

reasoning = {
    "effort": "low",  # 'low', 'medium', or 'high'
    "summary": "auto",  # 'detailed', 'auto', or None
}

llm = ChatOpenAI(model='gpt-4.1-nano')

examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 스티브 잡스는 몇 살에 사망했나요?
중간 답변: 스티브 잡스는 56세에 사망했습니다.
추가 질문: 아인슈타인은 몇 살에 사망했나요?
중간 답변: 아인슈타인은 76세에 사망했습니다.
최종 답변은: 아인슈타인
""",
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 네이버의 창립자는 누구인가요?
중간 답변: 네이버는 이해진에 의해 창립되었습니다.
추가 질문: 이해진은 언제 태어났나요?
중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
최종 답변은: 1967년 6월 22일
""",
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 율곡 이이의 어머니는 누구인가요?
중간 답변: 율곡 이이의 어머니는 신사임당입니다.
추가 질문: 신사임당은 언제 태어났나요?
중간 답변: 신사임당은 1504년에 태어났습니다.
추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
최종 답변은: 연산군
""",
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """이 질문에 추가 질문이 필요한가요: 예.
추가 질문: 올드보이의 감독은 누구인가요?
중간 답변: 올드보이의 감독은 박찬욱입니다.
추가 질문: 박찬욱은 어느 나라 출신인가요?
중간 답변: 박찬욱은 대한민국 출신입니다.
추가 질문: 기생충의 감독은 누구인가요?
중간 답변: 기생충의 감독은 봉준호입니다.
추가 질문: 봉준호는 어느 나라 출신인가요?
중간 답변: 봉준호는 대한민국 출신입니다.
최종 답변은: 예
""",
    },
]

example_prompt = PromptTemplate.from_template(
    "Question:\n{question}\nAnswer:\n{answer}"
)

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question:\n{question}\nAnswer:",
    input_variables=["question"],
)

question = "Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"

chain = prompt | llm | StrOutputParser()

res = chain.invoke({'question': question})

print(res)

## `langchain-hub`
다양한 사용자들이 업로드한 프롬프트를 받아서 활용

In [ ]:
from langchain import hub

prompt = hub.pull('hwchase17/react')
print(prompt.template)